In [ ]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
pd.set_option('display.max_rows', 20)

In [ ]:
EPL24_25=pd.read_excel("C:/Users/stefansz/Documents/SM451 Sports Analytics Fall 2024/Data/EPL results with odds 2024-25 to Jan 21.xlsx")
EPL24_25

# Calculating the bookmaker accuracy

In [ ]:
# Note that we calculate the most likely outcome based on the decimal odds, not the probability. 
# Since the decimal odds are just the inverse of the probability, the lowest value gives the most likely event

In [ ]:
EPL24_25['Avgres']= np.where((EPL24_25['AvgH']<EPL24_25['AvgD']) & (EPL24_25['AvgH']<=EPL24_25['AvgA']),'H',\
                            np.where((EPL24_25['AvgD']<EPL24_25['AvgH']) & (EPL24_25['AvgD']<EPL24_25['AvgA']),'D',\
                            np.where((EPL24_25['AvgA']<EPL24_25['AvgH']) & (EPL24_25['AvgA']<EPL24_25['AvgD']),'A',"")))
EPL24_25

In [ ]:
EPL24_25['Avgtrue']= np.where(EPL24_25['Avgres'] == EPL24_25['FTR'],1,0)
EPL24_25.describe(include='all')

In [ ]:
EPL24_25['winvalue'] = np.where(EPL24_25['FTR'] == 'H', 2 ,(np.where(EPL24_25['FTR'] == 'D', 1, 0)))
EPL24_25

In [ ]:
# Here we calculate the average number of correct predictions to Nov 4 
# See how we use .iloc to specify the rows we want to use

In [ ]:
EPL24_25['Avgtrue'].iloc[0:219].mean()

# Using the TM value ratio to predict results with an ordered logistic regression

In [ ]:
TMdat = pd.read_excel("C:/Users/stefansz/Documents/SM451 Sports Analytics Fall 2024/Data/EPL TM values 2011-12 to 2024-25.xlsx")
TMdat

In [ ]:
TMdat= TMdat.rename(columns={'TM value Sept 1 (euros)':'TMValue'})

In [ ]:
TMdat25 = TMdat[TMdat.seasonyrend == 2025].reset_index()
TMdat25

In [ ]:
TMdat25['HomeTeam']= TMdat25['Club']
TMdat25['AwayTeam']= TMdat25['Club']
TMdat25

In [ ]:
EPL24_25 = pd.merge(EPL24_25,TMdat25[['HomeTeam','TMValue']], on= 'HomeTeam', how = 'left').rename(columns={'TMValue':'hTM'})
EPL24_25

In [ ]:
EPL24_25 = pd.merge(EPL24_25,TMdat25[['AwayTeam','TMValue']], on= 'AwayTeam', how = 'left').rename(columns={'TMValue':'aTM'})
EPL24_25

In [ ]:
EPL24_25['lhTMratio'] = np.log(EPL24_25['hTM']/EPL24_25['aTM'])

In [ ]:
# this is the package for running ordered logistic regressions
import scipy.stats as stats
from statsmodels.miscmodels.ordinal_model import OrderedModel


In [ ]:
# this is logistic regression of win value on TM ratio
# This is the model used for the predictions last week - including results to Nov 4.
# see how we can again use .iloc to specify which rows of data we use for the regression

mod_Jan21 = OrderedModel(EPL24_25.iloc[0:219]['winvalue'],
                        EPL24_25.iloc[0:219][['lhTMratio']],
                        distr='logit')

res_Jan21 = mod_Jan21.fit(method='bfgs', disp=False)
res_Jan21.summary()

In [ ]:
#  PROBLEM
#
# There is a bug in the OrderedModel printout
# Below is the printout for the same regression run in Stata

In [ ]:
from IPython.display import Image

Image(filename='C:/Users/stefansz/Documents/SM451 Sports Analytics Fall 2024/Data/EPL Jan21 Stata logit.png')

In [ ]:
# PROBLEM

# Everything is the same apart from the value of the "1/2" or "/cut2" coefficient - the Stata value is the correct one
# There is no problem with the estimation itself, since the predicted values generated below by OrderedModel are correct
# When it comes to forecasting below, we will need to use the Stata coefficient (0.4805436), not the OrderedModel coefficient (0.2442)

In [ ]:
# Here we generate the predicted results from the regression model

predJan21 = res_Jan21.model.predict(res_Jan21.params, exog=None, offset=None, which='prob')
predJan21

In [ ]:
# the predictions come as an array. Here we convert the array into a df format so we can merge it with the EPL23_24 df

predJan21 = pd.DataFrame(predJan21,columns=("TM_pr(A)","TM_pr(D)","TM_pr(H)"))
predJan21

In [ ]:
# merge the predictions df into the games data
# note that we're merging two dataframes of different lengths, the first (EPL23_24) is the longer one,
# so we specify "left" so we get all the data.
# in the two dfs, each game has the same index, so we specify that they should merged based on the indexes


EPL24_25 =pd.merge(EPL24_25,predJan21, how='left',left_index=True, right_index=True)
EPL24_25

In [ ]:
# Now generate the outcome predictions based on the model

EPL24_25['Maxprob'] =EPL24_25[['TM_pr(A)','TM_pr(D)','TM_pr(H)']].max(axis=1)
EPL24_25['logitpred']=np.where(EPL24_25['Maxprob']==EPL24_25['TM_pr(A)'],'A',\
                               np.where(EPL24_25['Maxprob']==EPL24_25['TM_pr(D)'],'D','H'))
EPL24_25['logittrue']= np.where(EPL24_25['logitpred'] == EPL24_25['FTR'],1,0)
EPL24_25

In [ ]:
# We again use .iloc to specify the range of data - since we don't have predictions for game 101 onwards
# First our model's success rate

EPL24_25['logittrue'].iloc[0:219].mean()

In [ ]:
# Now remind ourselves of the bookmaker success rate

EPL24_25['Avgtrue'].iloc[0:219].mean()

In [ ]:
# Cases where the bookmaker and model generate the same prediction

EPL24_25['Avg=TM']= np.where(EPL24_25['Avgres'] == EPL24_25['logitpred'], 1,0)
EPL24_25['Avg=TM'].iloc[0:219].mean()

# Brier score comparison

In [ ]:
# Create a dummy variable for each of the three outcomes, depending on the result

EPL24_25['Houtcome']= np.where(EPL24_25['FTR']=='H',1,0)
EPL24_25['Doutcome']= np.where(EPL24_25['FTR']=='D',1,0)
EPL24_25['Aoutcome']= np.where(EPL24_25['FTR']=='A',1,0)
EPL24_25

In [ ]:
# now calculate the Brier score for our model...

Brierlogit = ((EPL24_25['TM_pr(H)'] - EPL24_25['Houtcome'])**2 +(EPL24_25['TM_pr(D)'] - EPL24_25['Doutcome'])**2 +\
             (EPL24_25['TM_pr(A)'] - EPL24_25['Aoutcome'])**2).iloc[0:219].mean()
Brierlogit

In [ ]:
# convert the bookmaker decimal odds to probabilities

EPL24_25['AvgHPr']= 1/(EPL24_25['AvgH'])/(1/(EPL24_25['AvgH'])+ 1/(EPL24_25['AvgD'])+ 1/(EPL24_25['AvgA']))
EPL24_25['AvgDPr']= 1/(EPL24_25['AvgD'])/(1/(EPL24_25['AvgH'])+ 1/(EPL24_25['AvgD'])+ 1/(EPL24_25['AvgA']))
EPL24_25['AvgAPr']= 1/(EPL24_25['AvgA'])/(1/(EPL24_25['AvgH'])+ 1/(EPL24_25['AvgD'])+ 1/(EPL24_25['AvgA']))
EPL24_25

In [ ]:
#...and generate the Brier score for the bookies

BrierAvg = ((EPL24_25['AvgHPr'] - EPL24_25['Houtcome'])**2 +(EPL24_25['AvgDPr'] - EPL24_25['Doutcome'])**2 +\
             (EPL24_25['AvgAPr'] - EPL24_25['Aoutcome'])**2).iloc[0:219].mean()


In [ ]:
BrierAvg

# Forecasting results

In [ ]:
pd.set_option('display.max_columns', None)
EPL24_25

In [ ]:
# Based on the data to Jan 21, we use the coefficients from the regression, combined with the values of the TM ratio,
# to generate the probabilities for the entire season
# Note: for games already played these are the same as the probabilities generated using the ".predict" command

EPL24_25['modAwin']= 1/(1+np.exp(.9964456*EPL24_25['lhTMratio']+.796036))
EPL24_25['modD']= 1/(1+np.exp(.9964456*EPL24_25['lhTMratio']-.4805436))-EPL24_25['modAwin']
EPL24_25['modHwin']= 1-EPL24_25['modAwin']-EPL24_25['modD']
EPL24_25

In [ ]:
print(EPL24_25.columns.tolist())

In [ ]:
# Here are the forecasts for Matchday 23 (Jan25-26)

Matchday23 = EPL24_25.iloc[219:229][['Date','HomeTeam', 'AwayTeam', 'hTM', 'aTM', 'lhTMratio', 'modAwin', 'modD', 'modHwin']]
Matchday23

In [ ]:
# derive prediction from probabilities based on the model

Matchday23['Maxprob'] =Matchday23[['modAwin','modD','modHwin']].max(axis=1)
Matchday23['logitpred']=np.where(Matchday23['Maxprob']==Matchday23['modAwin'],'A',\
                               np.where(Matchday23['Maxprob']==Matchday23['modD'],'D','H'))
Matchday23

In [ ]:
# These odds were copied from oddsportal.com on 1/22/2025

odds = np.array([['Bournemouth',2.07,3.61,3.41],['Brighton & Hove Albion',1.65,3.84,5.38],['Liverpool',1.1,11.66,19.73],['Manchester City',1.88,4.06,3.63],['Southampton',6.91,5.11,1.41],['Wolverhampton Wanderers',6.55,4.55,1.47],['Aston Villa',1.56,4.29,5.69],['Crystal Palace',2.08,3.66,3.36],['Fulham',2.31,3.45,3.04],['Tottenham Hotspur',1.46,5.19,5.65]])
odds

In [ ]:
odds = pd.DataFrame(odds,columns=("HomeTeam","OP(H)","OP(D)","OP(A)"))
odds

In [ ]:
Matchday23= pd.merge(Matchday23,odds, on= "HomeTeam", how = 'left')

In [ ]:
# derive prediction from probabilities based on the bookmaker odds
# note the decimal odds are just the inverse of the probability, so the lowest value gives the most likely event

Matchday23['MinOP'] =Matchday23[['OP(H)','OP(D)','OP(A)']].min(axis=1)
Matchday23['OPpred']=np.where(Matchday23['MinOP']==Matchday23['OP(A)'],'A',\
                               np.where(Matchday23['MinOP']==Matchday23['OP(D)'],'D','H'))
Matchday23

In [ ]:
column_to_move = Matchday23.pop("logitpred")
Matchday23.insert(14, "logitpred", column_to_move)
Matchday23

In [ ]:
Matchday23 = Matchday23.assign(Result = ['H','A','H','H','A','A','D','A','A','A'])
Matchday23

In [ ]:
Matchday24 = EPL24_25.iloc[229:239][['Date','HomeTeam', 'AwayTeam', 'hTM', 'aTM', 'lhTMratio', 'modAwin', 'modD', 'modHwin']]
Matchday24

In [ ]:
Matchday24['Maxprob'] =Matchday24[['modAwin','modD','modHwin']].max(axis=1)
Matchday24['logitpred']=np.where(Matchday24['Maxprob']==Matchday24['modAwin'],'A',\
                               np.where(Matchday24['Maxprob']==Matchday24['modD'],'D','H'))
Matchday24

In [ ]:
# These odds were copied from oddsportal.com on 1/22/2025

odds = np.array([['Arsenal',2.15,3.57,3.22],['Bournemouth',4.27,4.09,1.74],['Brentford',2.38,3.79,2.68],['Chelsea',1.37,5.24,7.22],['Everton',1.7,3.81,4.8],['Ipswich Town',1.94,3.57,3.82],['Manchester United',2.14,3.47,3.33],['Newcastle United',1.68,4.03,4.67],['Nottingham Forest',2.41,3.52,2.79],['Wolverhampton Wanderers',3.46,3.62,2.04]])
odds

In [ ]:
odds = pd.DataFrame(odds,columns=("HomeTeam","OP(H)","OP(D)","OP(A)"))
odds

In [ ]:
Matchday24= pd.merge(Matchday24,odds, on= "HomeTeam", how = 'left')
Matchday24

In [ ]:
# derive prediction from probabilities based on the bookmaker odds
# note the decimal odds are just the inverse of the probability, so the lowest value gives the most likely event

Matchday24['MinOP'] =Matchday24[['OP(H)','OP(D)','OP(A)']].min(axis=1)
Matchday24['OPpred']=np.where(Matchday24['MinOP']==Matchday24['OP(A)'],'A',\
                               np.where(Matchday24['MinOP']==Matchday24['OP(D)'],'D','H'))
Matchday24